<a href="https://colab.research.google.com/github/1997MarsRover/Small-LMs/blob/main/tokenization/BytePair-encoding/bpe_enc_dec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Here is Simple implementation and training of BPE

The *get_stats* func takes a vocabulary dictionary and computes the frequency of consecutive byte pairs across all words in the vocabulary.  

In [18]:
import requests
from bs4 import BeautifulSoup

url = "https://www.reedbeta.com/blog/reading-veach-thesis-2/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Extract the text content from specific HTML tags
article_text = soup.find("article").get_text()
print(article_text)



Reading Veach’s Thesis, Part 2
February 25, 2023 · Graphics, Math · Comments

In this post, we’re continuing to read Eric Veach’s doctoral thesis.
In our last installment, we covered the first half of the
thesis, dealing with theoretical foundations for Monte Carlo rendering. This time
we’re tackling chapters 8–9, including one of the key algorithms this thesis is famous for:
multiple importance sampling. Without further ado, let’s tuck in!
As before, this isn’t going to be a comprehensive review of everything in the thesis—it’s just a
selection of things that made me go “oh, that’s cool”, or “huh! I didn’t know that”.



Path-Space Integrals
Non-Local Path Sampling
Extended Light Path Expressions
Multiple Importance Sampling
The Balance Heuristic
The Power Heuristic
MIS Examples


Conclusion


Path-Space Integrals
We usually see the rendering equation expressed as a fixed-point integral equation. The radiance
field $L$ appears on both sides:
$$
    L = L_e + \int L \, f \, |\cos\the

In [30]:
tokens = one_line_text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens))

In [28]:
lines = [line.strip() for line in article_text.splitlines() if line.strip()]
one_line_text = ' '.join(lines)

In [29]:
one_line_text

'Reading Veach’s Thesis, Part\xa02 February 25, 2023 · Graphics, Math · Comments In this post, we’re continuing to read Eric Veach’s doctoral thesis. In our last installment, we covered the first half of the thesis, dealing with theoretical foundations for Monte Carlo rendering. This time we’re tackling chapters 8–9, including one of the key algorithms this thesis is famous for: multiple importance sampling. Without further ado, let’s tuck in! As before, this isn’t going to be a comprehensive review of everything in the thesis—it’s just a selection of things that made me go “oh, that’s cool”, or “huh! I didn’t know that”. Path-Space Integrals Non-Local Path Sampling Extended Light Path Expressions Multiple Importance Sampling The Balance Heuristic The Power Heuristic MIS Examples Conclusion Path-Space Integrals We usually see the rendering equation expressed as a fixed-point integral equation. The radiance field $L$ appears on both sides: $$ L = L_e + \\int L \\, f \\, |\\cos\\theta| \

The *merge_vocab* function merges a given byte pair into a new token in the vocabulary. It uses regular expressions to replace the byte pair with the merged token across all words in the vocabulary.

In [31]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

# ---
vocab_size = 276 # the desired final vocabulary size
num_merges = vocab_size - 256
ids = list(tokens) # copy so we don't destroy the original list

The *bpe* function is the main function that performs the Byte Pair Encoding algorithm. It takes a vocabulary dictionary and the desired number of merge operations (num_merges).

- In each iteration, it calls *get_stats* to find the most frequent byte pair.
- If there are no more pairs to merge, it breaks out of the loop.
- Otherwise, it finds the most frequent byte pair (best) and calls *merge* func to merge it into a new token, updating the vocabulary.
- After the specified number of merges, it returns the final merged vocabulary.

In [32]:
def bpe(ids, num_merges):
    merges = {} # (int, int) -> int
    for i in range(num_merges):
        stats = get_stats(ids)
        pair = max(stats, key=stats.get)
        idx = 256 + i
        print(f"merging {pair} into a new token {idx}")
        ids = merge(ids, pair, idx)
        merges[pair] = idx

    return merges



merged_vocab = bpe(ids, num_merges=10)

merging (101, 32) into a new token 256
merging (116, 104) into a new token 257
merging (115, 32) into a new token 258
merging (105, 110) into a new token 259
merging (116, 32) into a new token 260
merging (101, 114) into a new token 261
merging (111, 110) into a new token 262
merging (32, 97) into a new token 263
merging (32, 257) into a new token 264
merging (116, 105) into a new token 265


In [15]:
merged_vocab

{(101, 32): 256,
 (240, 159): 257,
 (226, 128): 258,
 (105, 110): 259,
 (115, 32): 260,
 (97, 110): 261,
 (116, 104): 262,
 (257, 133): 263,
 (257, 135): 264,
 (97, 114): 265}

In [33]:
print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 11308
ids length: 11308
compression ratio: 1.00X


In [35]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merged_vocab.items():
    vocab[idx] = vocab[p0] + vocab[p1]

#Decoding.

Given a string of integers in the range [0, vocab_size], what is the text?

In [36]:
def decode(ids):
    # given ids (list of integers), return Python string
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text

print(decode([97]))

a


#Encoding

Given a string what are the token?

In [37]:
def encode(text):
    #given python string. return tokens (list of integers)

    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merged_vocab.get(p, float("inf")))
        if pair not in merged_vocab:
            break
        idx = merged_vocab[pair]
        tokens = merge(tokens, pair, idx)

    return tokens

print(encode("g"))

[103]


In [38]:
print(decode(encode("being a programmer")))

being a programmer


#Using gpt2 Tokenizer

In [39]:
!pip install tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.7 MB/s eta 0:00:00


In [41]:
import tiktoken

# GPT-2 (does not merge spaces)
enc = tiktoken.get_encoding("gpt2")
print(enc.encode("    being a programmer!!!"))

# GPT-4 (merges spaces)
enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("    being a programmer!!!"))

[220, 220, 220, 852, 257, 24292, 10185]
[262, 1694, 264, 48888, 12340]


#Using GPT-4 Tokenizer

In [43]:

print(enc.encode("привет 👋 (hello in Russian!)"))
print(enc.decode(enc.encode("привет 👋 (hello in Russian!)")) == "привет 👋 (hello in Russian!)")
# match the above for your own tokenizer, and also implement a train() function

[8164, 2233, 28089, 8341, 62904, 233, 320, 15339, 304, 8690, 16715]
True


In [46]:
!git clone https://github.com/karpathy/minbpe.git
%cd minbpe

Cloning into 'minbpe'...
remote: Enumerating objects: 217, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 217 (delta 130), reused 187 (delta 120), pack-reused 1
Receiving objects: 100% (217/217), 333.18 KiB | 7.75 MiB/s, done.
Resolving deltas: 100% (130/130), done.
/content/minbpe


This class is a light wrapper around the RegexTokenizer (2, above) that exactly reproduces the tokenization of GPT-4 in the tiktoken library. The wrapping handles some details around recovering the exact merges in the tokenizer, and the handling of some unfortunate (and likely historical?) 1-byte token permutations.

In [47]:
from minbpe import GPT4Tokenizer

tokenizer = GPT4Tokenizer()
print(tokenizer.encode("привет 👋 (hello in Russian!)"))

[8164, 2233, 28089, 8341, 62904, 233, 320, 15339, 304, 8690, 16715]
